In [85]:
import pandas as pd,numpy as np,datetime,time,re,copy,pickle
import pyodbc as pyd
from dateutil.parser import parse
import os,sys,io_data
import importlib
importlib.reload(io_data)
from pdb import set_trace
from dateutil.parser import parse
import Redundant_func as Rf


io1 = io_data.io_d1()
io1.path = {"value":"data","typ":"part_name"}

私有属性path更新为: /Users/dt/Desktop/barra/data


### 沪深300+中证500股票池，剔除停牌，被st标的，动态池

##### 样本选择，对应行业分类标准

In [70]:
#A股+大盘指数
Code_name = pd.read_sql("""
select * from SecuMain 
where SecuMarket in (83,90)
and SecuCategory in (1,4)
and ListedSector in (1,6)
""",con = conn)
io1.upload(Code_name,"Code_name")

In [166]:
#复权信息下载
fuquan_factor = pd.read_sql("""
select a.ExDiviDate,b.SecuCode,a.AdjustingFactor,a.AdjustingConst
from QT_AdjustingFactor as a
left join SecuMain as b
on a.InnerCode = b.InnerCode 
where b.SecuCode in (%s)
and a.ExDiviDate > '2014-01-01'
"""%("'"+"','".join(valid_code)+"'"),con = conn)
io1.upload(fuquan_factor,"fuquan")

In [71]:
#指数基本信息
Index_info = pd.read_sql("""
select * from LC_IndexBasicInfo as a
left join SecuMain as b
on a.IndexCode = b.InnerCode
and a.ComponentType = 1000
""",con = conn)
io1.upload(Index_info,"Index_info")

In [87]:
#指数成分权重
HS_Component = pd.read_sql("""
select a1.SecuInnerCode,a1.SecuMarket,a1.InDate,a1.OutDate,
b1.SecuCode,b1.ChiName,b1.ChiNameAbbr,a1.SecuCode1
from (select a.SecuInnerCode,a.SecuMarket,a.InDate,a.OutDate,b.SecuCode as SecuCode1
from LC_IndexComponent as a
left join SecuMain as b
on a.IndexInnerCode = b.InnerCode
where b.SecuCode = '000300' 
) as a1
left join SecuMain as b1
on a1.SecuInnerCode = b1.InnerCode
and ((a1.OutDate is Null) or a1.OutDate > '2015-01-01')
""",con = conn)

ZZ_Component = pd.read_sql("""
select a1.SecuInnerCode,a1.SecuMarket,a1.InDate,a1.OutDate,
b1.SecuCode,b1.ChiName,b1.ChiNameAbbr,a1.SecuCode1
from (select a.SecuInnerCode,a.SecuMarket,a.InDate,a.OutDate,b.SecuCode as SecuCode1
from LC_IndexComponent as a
left join SecuMain as b
on a.IndexInnerCode = b.InnerCode
where b.SecuCode = '000905' 
) as a1
left join SecuMain as b1
on a1.SecuInnerCode = b1.InnerCode
and ((a1.OutDate is Null) or a1.OutDate > '2015-01-01')
""",con = conn)
io1.upload(HS_Component,"HS_comp")
io1.upload(ZZ_Component,"ZZ_comp")
#退市、st标的，沪深300，中证500标的区间不会出现退市、分红

In [88]:
#需要查找的公司标的
importlib.reload(Rf)
valid_code = Rf.extract_valid_company(pd.unique(HS_Component["SecuCode"]),\
                                     pd.unique(ZZ_Component["SecuCode"]),typ = "kechuang")

# io1.upload(valid_code,"valid_code_chuangye")

In [116]:
#行业信息
Industry_info = pd.read_sql("""
select b.SecuCode,a.InfoPublDate,a.Industry,a.FirstIndustryName,
a.SecondIndustryName,a.InsertTime
from LC_ExgIndustry as a
left join SecuMain as b
on a.CompanyCode = b.InnerCode
where b.SecuCode in (%s)
and a.Standard = 38
and a.IfPerformed = 1
"""%("'"+"','".join(valid_code)+"'"),con = conn)

Industry_info1 = pd.read_sql("""
select b.SecuCode,a.InfoPublDate,a.Industry,a.FirstIndustryName,
a.SecondIndustryName,a.InsertTime
from LC_ExgIndustry as a
left join SecuMain as b
on a.CompanyCode = b.InnerCode
where b.SecuCode in (%s)
and a.Standard = 24
and a.IfPerformed = 1
"""%("'"+"','".join(valid_code)+"'"),con = conn)
io1.upload(Industry_info,"Industry_info_new")
io1.upload(Industry_info1,"Industry_info_old")

In [126]:
#hs300指数收益率
hs300_price = pd.read_sql("""
select a.TradingDay,b.SecuCode,
a.PrevClosePrice,a.OpenPrice,a.HighPrice,a.LowPrice,a.ClosePrice,a.ChangePCT
from QT_CSIIndexQuote as a 
left join SecuMain as b
on a.IndexCode = b.InnerCode
where b.SecuCode = '000300'
and a.TradingDay > '2015-01-01'
""",con = conn)
io1.upload(hs300_price.sort_values("TradingDay"),"hs300_price")

In [138]:
#个股收益率
Individual_stock_ret = pd.read_sql("""
select a.TradingDay,b.SecuCode,a.PrevClosePrice,a.OpenPrice,a.HighPrice,a.LowPrice,a.ClosePrice,
a.ChangePCT,a.BetaHS300Index,a.BetaCompositeIndex,a.AlphaHS300Index,a.AlphaCompositeIndex
from QT_Performance as a
left join SecuMain as b
on a.InnerCode = b.InnerCode
where b.SecuCode in (%s)
and a.TradingDay > '2015-01-01'
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
"""%("'"+"','".join(valid_code)+"'"),con = conn)

io1.upload(Individual_stock_ret.sort_values(["SecuCode","TradingDay"]),"Individual_ret")

In [143]:
#总市值(万元)，成交量（万股）
TotalAsset_volume = pd.read_sql("""
select a.TradingDay,b.SecuCode,a.TurnoverVolume,a.TotalMV,a.NegotiableMV
from QT_Performance as a
left join SecuMain as b
on a.InnerCode = b.InnerCode
where b.SecuCode in (%s)
and a.TradingDay > '2015-01-01'
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
"""%("'"+"','".join(valid_code)+"'"),con = conn)

#成交量（万股）
io1.upload(TotalAsset_volume.sort_values(["SecuCode","TradingDay"]),"TotalAsset_volume")

In [90]:
#长期债务(元)，历史pe值
Long_Liability = pd.read_sql("""
select a.InfoPublDate,a.EndDate,b.SecuCode,a.TotalLongtermLiability
from LC_BalanceSheet as a
left join SecuMain as b
on a.CompanyCode = b.InnerCode
where b.SecuCode in (%s)
and a.InfoPublDate > '2015-01-01'
and a.EndDate > '2015-01-01'
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
and a.IfMerged = 1
"""%("'"+"','".join(valid_code)+"'"),con = conn)

PE = pd.read_sql("""
select a.TradingDay,b.SecuCode,a.PE
from LC_DIndicesForValuation as a
left join SecuMain as b
on a.InnerCode = b.InnerCode
where b.SecuCode in (%s)
and a.TradingDay > '2015-01-01'
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
"""%("'"+"','".join(valid_code)+"'"),con = conn)

In [150]:
io1.upload(Long_Liability.sort_values(["SecuCode"]),"Long_Liability")
io1.upload(PE.sort_values(["SecuCode","TradingDay"]),"PE")

In [251]:
#增加NetOperateCashFlow,NetInvestCashFlow,NetFinanceCashFlow,现金收益（元）
#OperatingReenue + NonoperatingIncome - NonoperatingExpense,营业总收入(元)
#所有者权益，过去5年归属母公司净利润，过去5年营业总收入
Cash_earnings = pd.read_sql("""
select a.InfoPublDate,a.EndDate,b.SecuCode,a.TotalShareholderEquity,
a.EPS*a.TotalShareholderEquity as Mu_profit,
(a.OperatingReenue + a.NonoperatingIncome - a.NonoperatingExpense) as All_Income,
(a.NetOperateCashFlow + a.NetInvestCashFlow + a.NetFinanceCashFlow) as cash_profit,
a.TotalShares
from LC_MainDataNew as a
left join SecuMain as b
on a.CompanyCode = b.CompanyCode
where b.SecuCode in (%s)
and a.InfoPublDate > '2012-01-01'
and a.EndDate > '2012-01-01'
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
and a.AccountingStandards = 1
and a.Mark = 1
"""%("'"+"','".join(valid_code)+"'"),con = conn)
io1.upload(Cash_earnings.sort_values(["SecuCode"]),"MainData")

In [155]:
#自由流通股本单位为股
Guben = pd.read_sql("""
select a.InfoPublDate,b.SecuCode,a.TotalAShare,a.FreeFloats,a.AdjFreeFloats
from LC_FreeFloat as a
left join SecuMain as b
on a.InnerCode = b.InnerCode
where b.SecuCode in (%s)
and a.InfoPublDate > '2015-01-01'
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
"""%("'"+"','".join(valid_code)+"'"),con = conn)

io1.upload(Guben.sort_values(["SecuCode"]),"Guben")

In [99]:
#总资产\总负债
TD_TA = pd.read_sql("""
select a.TradingDay,b.SecuCode,a.MarketValueToDebt1,a.TotalMV
from LC_DIndicesForValuation as a
left join SecuMain as b
on a.InnerCode = b.InnerCode
where b.SecuCode in (%s)
and a.TradingDay > '2015-01-01'
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
"""%("'"+"','".join(valid_code)+"'"),con = conn)

In [101]:
io1.upload(TD_TA,"TD_TA")

In [185]:
#上市时间
valid_code = io1.load("valid_code_chuangye")

In [196]:
ListStatus = pd.read_sql("""
select a.ChangeDate,b.SecuCode,a.ChangeType,a.Statement,a.ChangeReason
from LC_ListStatus as a
left join SecuMain as b
on a.InnerCode = b.InnerCode
where b.SecuCode in (%s)
and b.SecuCategory in (1)
and b.SecuMarket in (83,90)
"""%("'"+"','".join(valid_code)+"'"),con = conn)

In [3]:
t1 = io1.load("ListStatus")

In [10]:
pd.set_option("display.max_rows",500)
# t1[pd.Index(t1["ChangeType"]).isin([4])]

### csv一致预期文件

In [30]:
import warnings
warnings.filterwarnings("ignore")
SH_ind = pd.read_csv(os.path.join(os.getcwd(),"data","SH_code.csv"),header =1,index_col = 0,parse_dates = True)
SZ_ind = pd.read_csv(os.path.join(os.getcwd(),"data","SZcode_indicator.csv"),header = 1,index_col = 0,parse_dates = True)

In [67]:
valid_col1 = [name for name in SH_ind.columns if "Unname" not in name]
valid_col2 = [name for name in SZ_ind.columns if "Unname" not in name]
valid_col1_num = [SH_ind.columns.get_loc(name) for name in valid_col1]
valid_col2_num = [SZ_ind.columns.get_loc(name) for name in valid_col2]
#一预预测净利润2年CAGR->west_netprofit_CAGR
#一致预测每股收益->west_eps_FTM
#一致预测净利润同比->west_netprofit_YOY
SH_ind1 = pd.read_csv(os.path.join(os.getcwd(),"data","SH_code.csv"),header =3,index_col = 0,parse_dates = True)
SZ_ind1 = pd.read_csv(os.path.join(os.getcwd(),"data","SZcode_indicator.csv"),header = 3,index_col = 0,parse_dates = True)

In [76]:
netprofit_2Y_CAGR = pd.DataFrame([])
temp1 = SH_ind1.iloc[:,valid_col1_num[0]:valid_col1_num[1] - 2]
temp2 = SZ_ind1.iloc[:,valid_col2_num[0]:valid_col2_num[1] - 2]
netprofit_2Y_CAGR = pd.concat([temp2,temp1],axis = 1)
io1.upload(netprofit_2Y_CAGR,"netprofit_2Y_CAGR")

In [83]:
est_eps = pd.DataFrame([])
temp1 = SH_ind1.iloc[:,valid_col1_num[1]:valid_col1_num[2] - 2]
temp2 = SZ_ind1.iloc[:,valid_col2_num[1]:valid_col2_num[2] - 2]
est_eps = pd.concat([temp2,temp1],axis = 1)
io1.upload(est_eps,"est_eps")

In [84]:
netprofit_1Y_CAGR = pd.DataFrame([])
temp1 = SH_ind1.iloc[:,valid_col1_num[2]:]
temp2 = SZ_ind1.iloc[:,valid_col2_num[2]:]
netprofit_1Y_CAGR = pd.concat([temp2,temp1],axis = 1)
io1.upload(netprofit_1Y_CAGR,"netprofit_1Y_CAGR")